In [18]:
# !pip install rank-bm25

In [19]:
# 1. 초기 설정 및 Chroma DB 연결
import torch
import chromadb
from sentence_transformers import SentenceTransformer

# 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델 로드
model = SentenceTransformer("dragonkue/BGE-m3-ko").to(device)

# Chroma DB 연결
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_collection(name="patent_claims")

In [20]:
# 2. BM25 모델 생성
from rank_bm25 import BM25Okapi

documents = collection.get()["documents"]

# 토큰화(청구항)
tokenized_docs = [doc.split() for doc in documents]

# BM25 생성
bm25 = BM25Okapi(tokenized_docs)                

In [21]:
# 3. doc_id → BM25 인덱스 매핑
doc_id_to_idx = {}
raw_ids = collection.get()["ids"]  # 전체 ID 가져오기

for idx, doc_id in enumerate(raw_ids):
    doc_id_to_idx[doc_id] = idx

In [22]:
# 4. 하이브리드 서치 함수 정의
def hybrid_search(query, top_k=1000):

    if isinstance(query, list):
        results_list = []
        for q in query:
            results_list.append(hybrid_search(q, top_k))
        return results_list

    # 1) 벡터 검색
    query_vector = model.encode(query).tolist()

    results = collection.query(
        query_embeddings=[query_vector],
        n_results=top_k
    )

    vector_docs = results["documents"][0]
    vector_ids = results["ids"][0]  
    vector_metas = results["metadatas"][0]
    vector_scores = [1 - d for d in results["distances"][0]] 

    # 2) BM25 검색
    tokenized_query = query.split()
    bm25_scores = bm25.get_scores(tokenized_query) 

    # 3) 점수 합산
    hybrid_list = []
    for doc_text, meta, doc_id, v_score in zip(vector_docs, vector_metas, vector_ids, vector_scores):

        # BM25 점수 찾기 
        if doc_id in doc_id_to_idx:
            bm25_idx = doc_id_to_idx[doc_id]
            b_score = bm25_scores[bm25_idx]
        else:
            b_score = 0  

        final_score = 0.7 * v_score + 0.3 * b_score

        hybrid_list.append({
            "text": doc_text,
            "patent_id": meta.get("patent_id"),   
            "claim_no": meta.get("claim_no"),
            "score": final_score
        })


    # 4) 점수로 최종 정렬
    hybrid_list.sort(key=lambda x: x["score"], reverse=True)

    return hybrid_list


In [23]:
# 5. 하이브리드 서치 결과 확인
results = hybrid_search("영상 분석 3차원 정보 기반 컴퓨터비전 시스템")

for r in results:
    print(f"출원번호: {r['patent_id']}, 점수: {r['score']:.4f}")
    print(f"내용: {r['text'][:120]}...\n")

출원번호: 1020230132097, 점수: 6.7654
내용: 상기 프로세서는 상기 영상 분석 이벤트에 상응하는 CCTV의 식별정보 및 해당 영상의 URL 정보, 상기 영상 분석 이벤트의 시간 정보를 포함하는 영상 분석 이벤트를 저장하되,상기 영상 분석 이벤트의 시간 정보는 상...

출원번호: 1020250159707, 점수: 6.7107
내용: 차량에 설치된 전자 장치에 의해 수행되는, 상기 차량에 대한 위험 감지 방법에 있어서,상기 차량의 외부에 장착된 카메라를 이용하여 촬영된 원본 영상을 획득하는 동작;상기 원본 영상에 기초하여 딥러닝 영상 인식 방식에...

출원번호: 1020230132097, 점수: 6.7100
내용: 상기 복수의 CCTV로부터 수집된 영상을 대상으로 분석한 영상 분석 이벤트를 저장하는 단계는,상기 영상 분석 이벤트에 상응하는 CCTV의 식별정보 및 해당 영상의 URL 정보, 상기 영상 분석 이벤트의 시간 정보를 ...

출원번호: 1020240091382, 점수: 6.1476
내용: 상기 3차원 포인트의 정보는, 상기 3차원 포인트의 3차원 공간상의 위치 정보, 크기 정보, 점유도 정보 또는 방향별 색상 정보 중 적어도 하나를 포함하는 것을 특징으로 하는, 영상 부호화 방법....

출원번호: 1020240091382, 점수: 6.1400
내용: 상기 3차원 포인트의 정보는, 상기 3차원 정점의 3차원 공간상의 위치 정보, 크기 정보, 점유도 정보 또는 방향별 색상 정보 중 적어도 하나를 포함하는 것을 특징으로 하는, 영상 복호화 방법....

출원번호: 1020230171850, 점수: 6.0163
내용: (3)모니터링 시스템을 통하여 실시간 디바이스-서버 통신 산란수 측정 및 과산계 선별별과 가시화를 위한 통합 모니터링 구성함을 특징으로 하는 딥러닝 컴퓨터비전 기반 과산계 선별 시스템....

출원번호: 1020220161550, 점수: 5.9872
내용: 피험자의 영상을 수집하는 영상 획득 모듈과;수집된 영상을 컴퓨터비전 

---

##### 결과 테스트

- 출원번호를 뽑아서 정답 출원번호와 교집합 몇 개인지

In [24]:
# 같은 ipcNumber을 가진 출원번호
ipc_g05d143 = {"1020240014135",
"1020230114322",
"1020250071099",
"1020257016601",
"1020240039664",
"1020240037602",
"1020240024079",
"1020230147397",
"1020257029654",
"1020240062392",
"1020257011052",
"1020220110142",
"1020220132771",
"1020230173300",
"1020240085547",
"1020250040520",
"1020240013189",
"1020247037212",
"1020257009077",
"1020247018578",
"1020247018568",
"1020247018428",
"1020240079243",
"1020237031214",
"1020240026208",
"1020230177082",
"1020220120962",
"1020240003758",
"1020230134552",
"1020230127447",
"1020230127436"}

In [25]:
def eval(query, ipc_list, TOP_K):
    final_response = hybrid_search(query)

    # 비교
    # flatten
    flat_response = [item for sublist in final_response for item in sublist]

    response_patent_id = [f['patent_id'] for f in flat_response]

    print(ipc_list & set(response_patent_id))
    print("Precision:", len(ipc_list & set(response_patent_id)) / TOP_K)

In [26]:
query = [
    "주행 로봇의 위치·방향을 판단하고 제어하는 알고리즘",
    "지도 기반 위치 결정 및 경로 추종 제어 로직"
]

TOP_K = 100
eval(query, ipc_g05d143, TOP_K)

{'1020230127447', '1020257011052', '1020257029654', '1020247037212', '1020250040520', '1020240079243', '1020230134552', '1020257016601', '1020230127436', '1020220132771', '1020220120962', '1020240039664', '1020240003758'}
Precision: 0.13


---

##### 정답 데이터 수정 (G06T)

In [27]:
# RAG 구조 확인
results = collection.peek(1)
print(results)

{'ids': ['1020050108060_claim1'], 'embeddings': array([[ 0.01750535,  0.01169939, -0.06720747, ...,  0.03021857,
        -0.01640843, -0.00635329]], shape=(1, 1024)), 'documents': ['주변의 다른 적어도 하나의 단말기와 통신하며 정보를 교류하는 단말기로서,상기 다른 단말기와의 통신 기능을 수행하는 통신부;단말기의 식별정보를 포함한 정보를 저장하는 메모리;상기 통신부를 통해 상기 다른 단말기에 원하는 정보를 요청하고, 요청된 정보를 수신하여 해당 서비스를 수행하는 제어부를 포함하는 단말기.'], 'uris': None, 'included': ['metadatas', 'documents', 'embeddings'], 'data': None, 'metadatas': [{'claim_no': 1, 'ipc': 'H04M 3/42, H04B 1/40, G06F 17/00, G06Q 30/06', 'link': 'http://plus.kipris.or.kr/kiprisplusws/fileToss.jsp?arg=452306798ef1fd035bfb5ff666db4c9c592fb536ea920a4d00a52c7cb74f75b34e9968e7031a190dadda0b8843d15a1b', 'priority': '대한민국', 'register': '공개', 'title': '정보 교류 기능을 가진 단말기, 이를 이용한 정보 교류시스템 및 방법', 'patent_id': '1020050108060'}]}


In [28]:
# 출원번호 검색 함수 정의
def search_by_patent_id(patent_id, top_k=100):
    
    dummy_vector = [0.0] * 1024 

    results = collection.query(
        query_embeddings=[dummy_vector],
        where={"patent_id": patent_id},
        n_results=top_k,
        include=["documents", "metadatas", "distances"]
    )
    return results

# 실행
results = search_by_patent_id("1020200136606")
results


{'ids': [['1020200136606_claim1',
   '1020200136606_claim2',
   '1020200136606_claim3',
   '1020200136606_claim4',
   '1020200136606_claim5',
   '1020200136606_claim6',
   '1020200136606_claim7',
   '1020200136606_claim8',
   '1020200136606_claim9',
   '1020200136606_claim10',
   '1020200136606_claim11',
   '1020200136606_claim12',
   '1020200136606_claim13',
   '1020200136606_claim14',
   '1020200136606_claim15',
   '1020200136606_claim16',
   '1020200136606_claim17',
   '1020200136606_claim18',
   '1020200136606_claim19',
   '1020200136606_claim20']],
 'embeddings': None,
 'documents': [['타겟 CV 애플리케이션에 따라 하나 이상의 입력 이미지들로부터 지도 출력에 대한 추정을 수행하고, 상기 지도 출력 및 상기 지도 출력의 실측에 따라 지도 손실을 결정하는 지도 학습 시스템;상기 지도 출력 및 상기 하나 이상의 입력 이미지들에 따라 비지도 손실을 결정하는 비지도 학습 시스템;상기 지도 출력 및 상기 하나 이상의 입력 이미지들에 대응하는 약한 라벨에 따라 약한(weakly) 지도 손실을 결정하는 약한 지도 학습 시스템; 및상기 지도 손실, 상기 비지도 손실 및 상기 약한 지도 손실을 최적화하는 공동 최적화 프로그램을 포함하는, 컴퓨터 비전(CV) 훈련 시스템.',
   '상기 타겟 CV 애플리케이션은 단안 깊이 추정에 해당하고, 상기 지도 출력은 상기 하나 이상의 입력 이미지들 중 타겟 이미지로부터

In [29]:
# 정답 데이터 로드
import pandas as pd

data = pd.read_csv('./test_data.csv')

In [41]:
patent_ids = set(data['출원번호'].astype(str))  

In [42]:
# 확인
print(f"총 {len(patent_ids)}개의 출원번호")
print("샘플:", list(patent_ids)[:5])

총 500개의 출원번호
샘플: ['1020160113642', '1020190119715', '1020167025917', '1020170005801', '1020210025905']


In [31]:
def hs_eval(query, ipc_list, TOP_K):
    final_response = hybrid_search(query)

    # query가 리스트인 경우 flatten 필요
    if isinstance(query, list):
        flat_response = [item for sublist in final_response for item in sublist]
    else:
        flat_response = final_response

    response_patent_id = [f['patent_id'] for f in flat_response]

    print(ipc_list & set(response_patent_id))
    print("Precision:", len(ipc_list & set(response_patent_id)) / TOP_K)


In [ ]:
# 실제 반환되는 patent_id 확인
query = [
    "영상 처리 3D Mapping 시스템 소프트웨어 데이터 처리 MCU 제어부", 
    "컴퓨터 입력 장치 키보드 터치판 사용자 인터페이스 소프트웨어 시스템"
]

results = hybrid_search(query, top_k=10)

# 리스트인 경우 flatten
if isinstance(query, list):
    flat_response = [item for sublist in results for item in sublist]
else:
    flat_response = results

# 실제 patent_id 출력
print("=== 실제 반환된 patent_id 샘플 ===")
for i, r in enumerate(flat_response[:10]):
    print(f"{i+1}. patent_id: '{r['patent_id']}' (type: {type(r['patent_id'])})")

# test_data의 patent_id 샘플 출력
print("\n=== test_data.csv의 patent_id 샘플 ===")
for i, pid in enumerate(list(patent_ids)[:10]):
    print(f"{i+1}. patent_id: '{pid}' (type: {type(pid)})")


=== 실제 반환된 patent_id 샘플 ===
1. patent_id: '1020207026278' (type: <class 'str'>)
2. patent_id: '1020210140662' (type: <class 'str'>)
3. patent_id: '1020207026278' (type: <class 'str'>)
4. patent_id: '1020217039153' (type: <class 'str'>)
5. patent_id: '1020247015182' (type: <class 'str'>)
6. patent_id: '1020190106521' (type: <class 'str'>)
7. patent_id: '1020190106521' (type: <class 'str'>)
8. patent_id: '1020230161282' (type: <class 'str'>)
9. patent_id: '1020220113402' (type: <class 'str'>)
10. patent_id: '1020257001491' (type: <class 'str'>)

=== test_data.csv의 patent_id 샘플 ===
1. patent_id: '1020160113642' (type: <class 'str'>)
2. patent_id: '1020190119715' (type: <class 'str'>)
3. patent_id: '1020167025917' (type: <class 'str'>)
4. patent_id: '1020170005801' (type: <class 'str'>)
5. patent_id: '1020210025905' (type: <class 'str'>)
6. patent_id: '1020217033426' (type: <class 'str'>)
7. patent_id: '1019990052571' (type: <class 'str'>)
8. patent_id: '1020227025051' (type: <class 'str'>

In [ ]:
# 결과 확인
query = [
    "영상 처리 3D Mapping 시스템 소프트웨어 데이터 처리 MCU 제어부", 
    "컴퓨터 입력 장치 키보드 터치판 사용자 인터페이스 소프트웨어 시스템"
]

TOP_K = 100
hs_eval(query, patent_ids, TOP_K)

{'1020237027372', '1020257024391', '1020230155140', '1020230020514', '1020257006522', '1020230164362'}
Precision: 0.06
